In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup

#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.linear_model import LogisticRegressionCV
#from sklearn.model_selection import train_test_split, cross_val_score
#from sklearn.metrics import confusion_matrix
#from sklearn.model_selection import cross_val_predict
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.decomposition import LatentDirichletAllocation

In [2]:
main_url = 'http://dogtime.com/dog-breeds/'

In [3]:
#get list of links of all breeds from main page

breed_links = []

conn = urlopen(main_url)
html = conn.read()

soup = BeautifulSoup(html)
links = soup.find_all('a')

for tag in links:
    link = tag.get('href',None)
    if (link is not None) & ('dogtime.com/dog-breeds/' in link):
        if ('/characteristics/' not in link) & ('/groups' not in link):
            breed_links.append(link)

/Users/zac.robertson/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/zac.robertson/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
#quick deduplication
breed_links = set(breed_links)
breed_links = list(breed_links)

In [5]:
breed_links

['http://dogtime.com/dog-breeds/english-cocker-spaniel',
 'http://dogtime.com/dog-breeds/bernedoodle',
 'http://dogtime.com/dog-breeds/maltipoo',
 'http://dogtime.com/dog-breeds/chinese-crested',
 'http://dogtime.com/dog-breeds/rat-terrier',
 'http://dogtime.com/dog-breeds/komondor',
 'http://dogtime.com/dog-breeds/giant-schnauzer',
 'http://dogtime.com/dog-breeds/cardigan-welsh-corgi',
 'http://dogtime.com/dog-breeds/norwegian-buhund',
 'http://dogtime.com/dog-breeds/pyrenean-shepherd',
 'http://dogtime.com/dog-breeds/irish-red-and-white-setter',
 'http://dogtime.com/dog-breeds/keeshond',
 'http://dogtime.com/dog-breeds/pekingese',
 'http://dogtime.com/dog-breeds/otterhound',
 'http://dogtime.com/dog-breeds/clumber-spaniel',
 'http://dogtime.com/dog-breeds/shiba-inu',
 'http://dogtime.com/dog-breeds/norfolk-terrier',
 'http://dogtime.com/dog-breeds/puggle',
 'http://dogtime.com/dog-breeds/newfoundland',
 'http://dogtime.com/dog-breeds/dogue-de-bordeaux',
 'http://dogtime.com/dog-breed

In [17]:
def breed_parser(site):
    page = urlopen(site)
    soup = BeautifulSoup(page, 'lxml')
    features = soup.findAll('div', class_='star-by-breed')
    traits = []
    ratings = []
    breed = soup.find('h1').find(text=True)
    for feature in features:
        traits.append(feature.find('span', class_='characteristic').find(text=True))
        ratings.append(feature.find('span', class_='star').attrs['class'][1][-1])
    df = pd.DataFrame({
        'rating': ratings,
    }, index = traits).T
    df.index = [breed]
    
    meta_list = soup.findAll('meta')
    try:
        img = [link for link in meta_list if '.jpg' in str(link)][0]['content']
        df['image'] = img
    except:
        print('Error on ' + site)
    
    return df

In [18]:
breed_parser(breed_links[4])

,Adaptability,Adapts Well to Apartment Living,Good For Novice Owners,Sensitivity Level,Tolerates Being Alone,Tolerates Cold Weather,Tolerates Hot Weather,All Around Friendliness,Affectionate with Family,Incredibly Kid Friendly Dogs,...,Potential For Mouthiness,Prey Drive,Tendency To Bark Or Howl,Wanderlust Potential,Exercise Needs,Energy Level,Intensity,Exercise Needs,Potential For Playfulness,image
Rat Terrier,3,5,3,4,3,1,4,4,4,5,...,2,4,1,4,5,5,4,4,5,http://cdn2-www.dogtime.com/assets/uploads/201...


In [19]:
frames = [breed_parser(dog) for dog in breed_links]

Error on http://dogtime.com/dog-breeds/jindo


In [20]:
all_ratings = pd.concat(frames)

In [21]:
all_ratings.drop('Korean Jindo Dog', inplace=True) #incomplete data for this breed

In [22]:
all_ratings

,Adaptability,All Around Friendliness,Exercise Needs,Health Grooming,Trainability,Adapts Well to Apartment Living,Affectionate with Family,Amount Of Shedding,Dog Friendly,Drooling Potential,...,Potential For Weight Gain,Prey Drive,Sensitivity Level,Size,Tendency To Bark Or Howl,Tolerates Being Alone,Tolerates Cold Weather,Tolerates Hot Weather,Wanderlust Potential,image
English Cocker Spaniel,4,5,4,3,4,5,5,3,5,1,...,5,5,5,2,4,1,4,3,4,http://cdn1-www.dogtime.com/assets/uploads/201...
Bernedoodle,3,4,4,3,4,3,5,1,4,2,...,3,3,4,3,3,2,5,3,3,http://cdn3-www.dogtime.com/assets/uploads/201...
Maltipoo,4,4,3,2,4,5,5,1,4,1,...,2,2,5,2,5,1,2,3,4,http://cdn3-www.dogtime.com/assets/uploads/201...
Chinese Crested,3,4,2,2,3,5,4,2,4,1,...,2,3,5,2,3,1,1,3,1,http://cdn1-www.dogtime.com/assets/uploads/201...
Rat Terrier,3,4,5,3,3,5,4,4,3,1,...,3,4,4,1,1,3,1,4,4,http://cdn2-www.dogtime.com/assets/uploads/201...
Komondor,3,3,3,3,4,2,5,3,1,2,...,3,2,4,4,5,1,4,3,4,http://cdn1-www.dogtime.com/assets/uploads/201...
Giant Schnauzer,2,3,4,3,4,1,5,4,3,1,...,3,4,4,4,4,2,4,2,4,http://cdn2-www.dogtime.com/assets/uploads/201...
Cardigan Welsh Corgi,4,4,3,3,3,4,5,4,3,1,...,4,4,4,2,2,3,4,3,3,http://cdn3-www.dogtime.com/assets/uploads/201...
Norwegian Buhund,3,4,5,3,3,3,5,3,3,1,...,3,2,3,3,4,4,5,2,4,http://cdn2-www.dogtime.com/assets/uploads/201...
Pyrenean Shepherd,3,4,5,2,3,5,5,3,2,1,...,3,4,4,2,5,1,4,3,2,http://cdn3-www.dogtime.com/assets/uploads/201...


In [23]:
all_ratings.to_csv('all_ratings.csv')